In [1]:

import tensorflow as tf
print(tf.__version__)


2.16.1


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths
input_dir = r"C:\Users\net pc\Desktop\Project with Dr. Mumbi\LB_CC_22.4"
label_dir = r"C:\Users\net pc\Desktop\Target\LB_CC_22.4"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")



Data loaded successfully!


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class InstanceNormalization(Layer):
    def __init__(self, epsilon=1e-5):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale',
                                     shape=input_shape[-1:],
                                     initializer=tf.random_normal_initializer(1., 0.02),
                                     trainable=True)
        self.offset = self.add_weight(name='offset',
                                      shape=input_shape[-1:],
                                      initializer='zeros',
                                      trainable=True)

    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (inputs - mean) * inv
        return self.scale * normalized + self.offset


In [10]:
from keras.layers import Conv2D, Input, Concatenate, LeakyReLU, Activation
from keras.initializers import RandomNormal
from keras.models import Model

def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = InstanceNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam')
   
    return model


In [11]:
from tensorflow.keras.layers import PReLU

def define_generator(image_shape=(256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
    e1 = LeakyReLU(alpha=0.2)(e1)
    
    e2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e1)
    e2 = BatchNormalization()(e2)
    e2 = LeakyReLU(alpha=0.2)(e2)
    
    e3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e2)
    e3 = BatchNormalization()(e3)
    e3 = LeakyReLU(alpha=0.2)(e3)
    
    # Bottleneck
    b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e3)
    b = PReLU(shared_axes=[1, 2])(b)  # Use PReLU from tf.keras.layers
    
    # Decoder
    d1 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(b)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)
    d1 = Concatenate()([d1, e3])
    
    d2 = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d1)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)
    d2 = Concatenate()([d2, e2])
    
    d3 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d2)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)
    d3 = Concatenate()([d3, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d3)
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [12]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return [X1, X2], y


In [13]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [14]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    
    # Input tensor for the generator model
    gen_input = Input(shape=image_shape)
    
    # Connect the generator output to the discriminator input
    gan_output = d_model([gen_input, g_model(gen_input)])
    
    # Define GAN model taking generator input and outputting discriminator decision
    gan_model = Model(gen_input, gan_output)
    
    # Compile GAN model
    gan_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    
    return gan_model

# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)


C:\Users\Public\Efendim\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [15]:

import numpy as np

def train(d_model, g_model, gan_model, dataset, n_epochs=10000, n_batch=1):
    # Determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # Unpack dataset
    trainA, trainB = dataset
    # Calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)  # Convert to integer
    # Calculate the total number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        # Select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # Generate a batch of fake samples
        X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_patch)
        # Update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], np.ones((n_batch, n_patch, n_patch, 1)))
        # Update discriminator for fake samples
         # Update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], np.zeros((n_batch, n_patch, n_patch, 1)))
        # Update the generator
        g_loss = gan_model.train_on_batch(X_realA, [np.ones((n_batch, n_patch, n_patch, 1)), X_realB])
        # Summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # Summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)



In [16]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 
import os
from PIL import Image

from PIL import Image


def save_img(filename, image):
    """Save the image to the specified filename."""
    # Scale the image back to the range [0, 255] before saving
    scaled_image = (image * 255).astype(np.uint8)
    # Convert to PIL Image
    im = Image.fromarray(scaled_image)
    # Save the image
    im.save(filename)
    
# Summarize performance
def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # Select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # Generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # Scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    

    output_path = "C:\\Users\\net pc\\Desktop\\generated\\LB_CC_22.4"

    for i in range(n_samples):
        # Save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # Save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # Save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # Save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")

    


In [ ]:
# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=10000, n_batch=1)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


C:\Users\Public\Efendim\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


>1, d1[0.681] d2[0.870] g[0.693]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>2, d1[0.807] d2[0.870] g[0.816]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>3, d1[0.832] d2[0.870] g[0.809]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
>4, d1[0.843] d2[0.870] g[0.729]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
>5, d1[0.849] d2[0.870] g[0.662]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
>6, d1[0.853] d2[0.871] g[0.600]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
>7, d1[0.856] d2[0.871] g[0.548]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
>8, d1[0.858] d2[0.871] g[0.501]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
>9, d1[0.860] d2[0.871] g[0.461]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>10, d1[0.861] d2[0.872] g[0.427]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
>11, d1[0.863] d2[0.872] g[0.398]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
>12, d1[0.864] d2[0.872] g[0.372]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>13, d1[0.865] d2[0.872] g[0.350]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>14, d1[0.865] d2[0.873] g[0.331]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
>15, d1[0.866] d2[0.873] g[0.314]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>16, d1[0.867] d2[0.873] g[0.298]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
>17, d1[0.867] d2[0.873] g[0.284]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
>18, d1[0.868] d2[0.873] g[0.271]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
>19, d1[0.868] d2[0.873] g[0.260]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
>20, d1[0.869] d2[0.874] g[0.249]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
>21, d1[0.869] d2[0.874] g[0.240]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
>22, d1[0.869] d2[0.874] g[0.231]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
>23, d1[0.870] d2[0.874] g[0.223]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>24, d1[0.870] d2[0.874] g[0.215]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
>25, d1[0.870] d2[0.874] g[0.208]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>26, d1[0.871] d2[0.874] g[0.202]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
>27, d1[0.871] d2[0.875] g[0.196]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
>28, d1[0.871] d2[0.875] g[0.190]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
>29, d1[0.871] d2[0.875] g[0.185]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
>30, d1[0.871] d2[0.875] g[0.180]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


...

Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
>81, d1[0.878] d2[0.879] g[0.079]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
>82, d1[0.878] d2[0.879] g[0.078]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>83, d1[0.878] d2[0.879] g[0.077]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>84, d1[0.878] d2[0.879] g[0.077]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
>85, d1[0.878] d2[0.880] g[0.076]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
>86, d1[0.878] d2[0.880] g[0.075]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
>87, d1[0.879] d2[0.880] g[0.075]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
>88, d1[0.879] d2[0.880] g[0.074]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
>89, d1[0.879] d2[0.880] g[0.073]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
>90, d1[0.879] d2[0.880] g[0.072]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step


Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
>91, d1[0.879] d2[0.880] g[0.072]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
>92, d1[0.879] d2[0.880] g[0.071]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
>93, d1[0.879] d2[0.880] g[0.070]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
>94, d1[0.879] d2[0.880] g[0.070]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>95, d1[0.879] d2[0.880] g[0.069]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
>96, d1[0.879] d2[0.881] g[0.069]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
>97, d1[0.880] d2[0.881] g[0.068]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
>98, d1[0.880] d2[0.881] g[0.067]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
>99, d1[0.880] d2[0.881] g[0.067]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
>100, d1[0.880] d2[0.881] g[0.066]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
>101, d1[0.880] d2[0.881] g[0.066]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
>102, d1[0.880] d2[0.881] g[0.065]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
>103, d1[0.880] d2[0.881] g[0.065]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
>104, d1[0.880] d2[0.881] g[0.064]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
>105, d1[0.880] d2[0.882] g[0.064]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>106, d1[0.881] d2[0.882] g[0.063]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
>107, d1[0.881] d2[0.882] g[0.063]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
>108, d1[0.881] d2[0.882] g[0.062]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>109, d1[0.881] d2[0.882] g[0.062]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
>110, d1[0.881] d2[0.882] g[0.061]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>111, d1[0.881] d2[0.882] g[0.061]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
>112, d1[0.881] d2[0.882] g[0.060]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>113, d1[0.881] d2[0.882] g[0.060]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
>114, d1[0.882] d2[0.883] g[0.059]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>115, d1[0.882] d2[0.883] g[0.059]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
>116, d1[0.882] d2[0.883] g[0.059]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>117, d1[0.882] d2[0.883] g[0.058]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
>118, d1[0.882] d2[0.883] g[0.058]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
>119, d1[0.882] d2[0.883] g[0.057]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
>120, d1[0.882] d2[0.883] g[0.057]
Saving images...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


Images and model saved at C:\Users\net pc\Desktop\generated\LB_CC_22.4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
>121, d1[0.883] d2[0.883] g[0.057]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
>122, d1[0.883] d2[0.884] g[0.056]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
>123, d1[0.883] d2[0.884] g[0.056]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
>124, d1[0.883] d2[0.884] g[0.055]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
